In [ ]:
import torch
torch.__version__


'1.13.1+cu116'

In [ ]:
torch.cuda.is_available()

True

In [ ]:
%%capture
!pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.5'

In [ ]:
%%capture
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch, torchvision
import matplotlib.pyplot as plt
import json
import cv2
import numpy as np
from copy import deepcopy
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.structures.image_list import ImageList
from detectron2.data import transforms as T
from detectron2.modeling.box_regression import Box2BoxTransform
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers
from detectron2.structures.boxes import Boxes
from detectron2.layers import nms
from detectron2 import model_zoo
from detectron2.config import get_cfg

import torch.utils.data as data
from torchvision.transforms import transforms
from transformers import BertTokenizer, VisualBertForVisualReasoning
from pycocotools.coco import COCO
import os

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import VisualBertModel
import random
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Define the data directory and annotation file paths

data_dir_c3d = '/content/gdrive/My Drive/Colab Notebooks/copy_rp/10000 relations/craft3d/doga_rel/'
raw_data_c3d = '/content/gdrive/My Drive/Colab Notebooks/copy_rp/10000 relations/craft3d/c3d_data.jsonl'

data_dir_vsr = '/content/gdrive/My Drive/Colab Notebooks/copy_rp/10000 relations/vsr/vsr_images/'
raw_data_vsr = '/content/gdrive/My Drive/Colab Notebooks/copy_rp/10000 relations/vsr/vsr_data.jsonl'

data_c3d = []
data_vsr = []


with open(raw_data_c3d, 'r') as json_file:
    json_list_c = list(json_file)

for json_str in json_list_c:
    result_c = json.loads(json_str)
    data_c3d.append(result_c)

with open(raw_data_vsr, 'r') as json_file:
    json_list_v = list(json_file)

for json_str in json_list_v:
    result_v = json.loads(json_str)
    data_vsr.append(result_v)   
    
# Step 1: Prepare the data
random.shuffle(data_c3d)
random.shuffle(data_vsr)

In [ ]:
#train_data = data[:int(0.0055*len(data))]
#valid_data = data[int(0.9946*len(data)):]
train_data_vsr = data_vsr[:80]
valid_data_vsr = data_vsr[len(data_vsr)-80:]

train_data_c3d = data_c3d[:80]
valid_data_c3d = data_c3d[len(data_c3d)-80:]


6940
4720
80
80
80
80
6940
4720
80
80
80
80


In [ ]:
def colorlist(data_dir, data):
  imageList = [data_dir+data[i]["image"] for i in range(len(data))]
  imageList = [cv2.cvtColor(plt.imread(n),cv2.COLOR_RGB2BGR) for n in imageList]
  #imageList = [cv2.resize(n, (1024,1024), interpolation = cv2.INTER_AREA) for n in imageList]
  #print(imageList[0].shape)
  #imageList = [cv2.resize(n, (200,200), interpolation = cv2.INTER_AREA) for n in imageList]
  return imageList

In [ ]:
cfg_path = "COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"

def load_config_and_model_weights(cfg_path):
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(cfg_path))

    # ROI HEADS SCORE THRESHOLD
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

    # Comment the next line if you're using 'cuda'
    #cfg['MODEL']['DEVICE']='cpu'

    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(cfg_path)

    return cfg


In [ ]:
def get_model(cfg):
    # build model
    model = build_model(cfg)

    # load weights
    checkpointer = DetectionCheckpointer(model)
    checkpointer.load(cfg.MODEL.WEIGHTS)

    # eval mode
    model.eval()
    return model


In [ ]:
def get_train_model(cfg):
    model = build_model(cfg)
    model.train()    
    return model

In [ ]:
def prepare_image_inputs(cfg, model, img_list):
    # Resizing the image according to the configuration
    
    transform_gen = T.ResizeShortestEdge(
                [400,400], cfg.INPUT.MAX_SIZE_TEST #cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST
            )
    
    print(img_list[0].shape)
    img_list = [transform_gen.get_transform(img).apply_image(img) for img in img_list]
    
    
    for k in img_list:
        print(k.shape)

    # Convert to C,H,W format
    convert_to_tensor = lambda x: torch.Tensor(x.astype("float32").transpose(2, 0, 1))

    batched_inputs = [{"image":convert_to_tensor(img), "height": img.shape[0], "width": img.shape[1]} for img in img_list]

    # Normalizing the image
    num_channels = len(cfg.MODEL.PIXEL_MEAN)
    pixel_mean = torch.Tensor(cfg.MODEL.PIXEL_MEAN).view(num_channels, 1, 1)
    pixel_std = torch.Tensor(cfg.MODEL.PIXEL_STD).view(num_channels, 1, 1)
    normalizer = lambda x: (x - pixel_mean) / pixel_std
    images = [normalizer(x["image"]).cuda() for x in batched_inputs]

    # Convert to ImageList
    images =  ImageList.from_tensors(images,model.backbone.size_divisibility)
    tensorx = images.tensor

    # You can then use the ".size()" method of the tensor to get the shape of the tensor
    # The shape of the tensor will represent the dimensions of the images in the ImageList
    print("img shape:",tensorx.size())

    return images, batched_inputs


In [ ]:
def get_features(model, images):
    features = model.backbone(images.tensor)
    return features

In [ ]:
def get_proposals(model, images, features):
    proposals, _ = model.proposal_generator(images, features)
    print("proposals:",len(proposals))
    return proposals


In [ ]:
def get_box_features(model, features, proposals, length):
    features_list = [features[f] for f in ['p2', 'p3', 'p4', 'p5']]
    box_features = model.roi_heads.box_pooler(features_list, [x.proposal_boxes for x in proposals])
    print("features:",len(features_list))
    print("features elem:",features_list[0].shape)
    print("prop:",len(proposals))
    print("after model roi heads:",box_features.shape)
    print(box_features.shape)
    box_features = model.roi_heads.box_head.flatten(box_features)
    print(box_features.shape)
    box_features = model.roi_heads.box_head.fc1(box_features)
    print(box_features.shape)
    num_proposals = min(box_features.shape[0]//length,1000) #?
    box_features = model.roi_heads.box_head.fc_relu1(box_features)
    print(box_features.shape)
    box_features = model.roi_heads.box_head.fc2(box_features)
    print(box_features.shape)
    box_features = box_features.reshape(length, num_proposals, 1024) # depends on your config and batch size
    return box_features, features_list


In [ ]:
def get_prediction_logits(model, features_list, proposals):
    cls_features = model.roi_heads.box_pooler(features_list, [x.proposal_boxes for x in proposals])
    cls_features = model.roi_heads.box_head(cls_features)
    pred_class_logits, pred_proposal_deltas = model.roi_heads.box_predictor(cls_features)
    return pred_class_logits, pred_proposal_deltas
    

In [ ]:
#from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputs

class FastRCNNOutputs:
    
    def __init__(
        self,
        box2box_transform,
        pred_class_logits,
        pred_proposal_deltas,
        proposals,
        smooth_l1_beta=0.0,
        box_reg_loss_type="smooth_l1",
    ):

        self.box2box_transform = box2box_transform
        self.num_preds_per_image = [len(p) for p in proposals]
        self.pred_class_logits = pred_class_logits
        self.pred_proposal_deltas = pred_proposal_deltas
        self.smooth_l1_beta = smooth_l1_beta
        self.box_reg_loss_type = box_reg_loss_type

        self.image_shapes = [x.image_size for x in proposals]

        #concatenation
        if len(proposals):
            box_type = type(proposals[0].proposal_boxes)
            # cat(..., dim=0) concatenates over all images in the batch
            self.proposals = box_type.cat([p.proposal_boxes for p in proposals])
            assert (
                not self.proposals.tensor.requires_grad
            ), "Proposals should not require gradients!"

            if proposals[0].has("gt_classes"):
                self.gt_classes = cat([p.gt_classes for p in proposals], dim=0)
                
                gt_boxes = [
                    p.gt_boxes if p.has("gt_boxes") else p.proposal_boxes for p in proposals
                ]
                self.gt_boxes = box_type.cat(gt_boxes)
        else:
            self.proposals = Boxes(torch.zeros(0, 4, device=self.pred_proposal_deltas.device))
        self._no_instances = len(self.proposals) == 0  # no instances found
        

    def predict_boxes(self):
        #Deprecated
        pred = self.box2box_transform.apply_deltas(self.pred_proposal_deltas, self.proposals.tensor)
        return pred.split(self.num_preds_per_image, dim=0)

    def predict_probs(self):
        #Deprecated
        probs = torch.nn.functional.softmax(self.pred_class_logits, dim=-1)
        return probs.split(self.num_preds_per_image, dim=0)


In [ ]:
def get_box_scores(cfg, pred_class_logits, pred_proposal_deltas, proposals):
    box2box_transform = Box2BoxTransform(weights=cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_WEIGHTS)
    smooth_l1_beta = cfg.MODEL.ROI_BOX_HEAD.SMOOTH_L1_BETA

    outputs = FastRCNNOutputs(
        box2box_transform,
        pred_class_logits,
        pred_proposal_deltas,
        proposals,
        smooth_l1_beta,
    )
    

    boxes = outputs.predict_boxes()
    scores = outputs.predict_probs()
    image_shapes = outputs.image_shapes

    return boxes, scores, image_shapes

In [ ]:
def get_output_boxes(boxes, batched_inputs, image_size):
    proposal_boxes = boxes.reshape(-1, 4).clone()
    scale_x, scale_y = (batched_inputs["width"] / image_size[1], batched_inputs["height"] / image_size[0])
    output_boxes = Boxes(proposal_boxes)

    output_boxes.scale(scale_x, scale_y)
    output_boxes.clip(image_size)

    return output_boxes

In [ ]:
def select_boxes(cfg, output_boxes, scores):
    test_score_thresh = cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST
    test_nms_thresh = cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST
    cls_prob = scores.detach()
    cls_boxes = output_boxes.tensor.detach().reshape(1000,80,4)
    max_conf = torch.zeros((cls_boxes.shape[0])).cuda()
    for cls_ind in range(0, cls_prob.shape[1]-1):
        cls_scores = cls_prob[:, cls_ind+1]
        det_boxes = cls_boxes[:,cls_ind,:]
        #keep = np.array(nms(det_boxes, cls_scores, test_nms_thresh))
        keep = nms(det_boxes, cls_scores, test_nms_thresh)
        max_conf[keep] = torch.where(cls_scores[keep] > max_conf[keep], cls_scores[keep], max_conf[keep])
    keep_boxes = torch.where(max_conf >= test_score_thresh)[0]
    return keep_boxes, max_conf

In [ ]:
def filter_boxes(keep_boxes, max_conf, min_boxes, max_boxes):
    if len(keep_boxes) < min_boxes:
        keep_boxes = np.argsort(max_conf.detach().cpu()).numpy()[::-1][:min_boxes]
    elif len(keep_boxes) > max_boxes:
        keep_boxes = np.argsort(max_conf.detach().cpu()).numpy()[::-1][:max_boxes]
    return keep_boxes

In [ ]:
def visual_embeds(box_features, keep_boxes):
    #print(keep_boxes.copy())
    #print(box_features[min(max(keep_boxes), len(box_features)-1)])
    return box_features[keep_boxes.copy()] #min(max(keep_boxes), len(box_features)-1)



In [ ]:
def get_visual_embeds(cfg_path,data_dir,data):
  cfg = load_config_and_model_weights(cfg_path)
  model = get_model(cfg)
  img_list = colorlist(data_dir, data)
  images, batched_inputs = prepare_image_inputs(cfg, model, img_list)
  features = get_features(model, images)
  proposals = get_proposals(model, images, features)
  box_features, features_list = get_box_features(model, features, proposals, len(img_list))
  
  pred_class_logits, pred_proposal_deltas = get_prediction_logits(model, features_list, proposals)
  boxes, scores, image_shapes = get_box_scores(cfg, pred_class_logits, pred_proposal_deltas, proposals)
  output_boxes = [get_output_boxes(boxes[i], batched_inputs[i], proposals[i].image_size) for i in range(len(proposals))]

  temp = [select_boxes(cfg, output_boxes[i], scores[i]) for i in range(len(scores))]
  keep_boxes, max_conf = [],[]
  for keep_box, mx_conf in temp:
      keep_boxes.append(keep_box)
      max_conf.append(mx_conf)
  
  MIN_BOXES=10
  MAX_BOXES=100

  keep_boxes = [filter_boxes(keep_box, mx_conf, MIN_BOXES, MAX_BOXES) for keep_box, mx_conf in zip(keep_boxes, max_conf)]

  the_visual_embeds = [visual_embeds(box_feature, keep_box).detach().cpu() for box_feature, keep_box in zip(box_features, keep_boxes)]
  the_visual_embeds = torch.stack(the_visual_embeds)

  del images, batched_inputs, features, proposals, box_features, pred_class_logits, pred_proposal_deltas, features_list

  return the_visual_embeds

In [ ]:
import os
from getpass import getpass
import urllib
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint
from tqdm.auto import tqdm

In [ ]:
def visualbert_train(model, train_data, valid_data, visual_embeds_train, visual_embeds_val, batch_size):

    val_best_score=0
    
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    criterion = nn.CrossEntropyLoss()

    model.cuda()
    model.train()

    for i in tqdm(np.arange(0, len(visual_embeds_train))):

        optimizer.zero_grad()

        img_attention_mask = torch.ones(visual_embeds_train[i].shape[:-1], dtype=torch.long) #.repeat(11, 1).t()
        img_token_type_ids = torch.ones(visual_embeds_train[i].shape[:-1], dtype=torch.long) #.repeat(11, 1).t()

        input_ids = [tokenizer.encode(m["caption"], add_special_tokens=True) for m in train_data[i:i+batch_size]]
        max_length = max(len(ids) for ids in input_ids)
        input_ids = [ids + [tokenizer.pad_token_id] * (max_length - len(ids)) for ids in input_ids]


        # Create the input dictionary
        inputs = {
            "input_ids": torch.tensor(input_ids).cuda()
        }

        inputs.update({
            "visual_embeds": visual_embeds_train[i].cuda(),
            "visual_token_type_ids": img_token_type_ids.cuda(),
            "visual_attention_mask": img_attention_mask.cuda(),
            })

        labels = [int(m["label"]) for m in train_data[i:i+batch_size]]

        labels = torch.tensor(labels).cuda() #.unsqueeze(0)

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        scores = outputs.logits
        out = F.softmax(scores,dim = 1)

        loss.backward(retain_graph=True) #retain_graph=True
        torch.cuda.empty_cache()
        optimizer.step()

        print(f"Loss for batch {i}: {loss.item()}")

        # evaluate
        acc = visualbert_eval(model, valid_data, visual_embeds_val, batch_size)
        print("New accuracy value for validation batch for train:",acc)
        if val_best_score < acc:
            val_best_score = acc
            print("NEW BEST ACCURACY:",val_best_score)
        else:
            continue
        
        checkpoint_dir = os.path.join('/content/gdrive/My Drive/Colab Notebooks/copy_rp/tmp/', f"best_checkpoint")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
            model.save_pretrained(checkpoint_dir)

    print("LAST BEST ACCURACY:",val_best_score)   
        

In [ ]:
def visualbert_eval(model, valid_data, visual_embeds_val, batch_size):

    correct, total = 0, 0

    model.cuda()
    model.eval()
    #model = VisualBertForVisualReasoning.from_pretrained(args.checkpoint_path)

    for i in tqdm(np.arange(0, len(visual_embeds_val))):

        img_attention_mask = torch.ones(visual_embeds_val[i].shape[:-1], dtype=torch.long) #.repeat(11, 1).t()
        img_token_type_ids = torch.ones(visual_embeds_val[i].shape[:-1], dtype=torch.long) #.repeat(11, 1).t()

        input_ids = [tokenizer.encode(m["caption"], add_special_tokens=True) for m in valid_data[i:i+batch_size]]
        max_length = max(len(ids) for ids in input_ids)
        input_ids = [ids + [tokenizer.pad_token_id] * (max_length - len(ids)) for ids in input_ids]


        # Create the input dictionary
        inputs = {
            "input_ids": torch.tensor(input_ids).cuda()
        }

        inputs.update({
            "visual_embeds": visual_embeds_val[i].cuda(),
            "visual_token_type_ids": img_token_type_ids.cuda(),
            "visual_attention_mask": img_attention_mask.cuda(),
            })

        labels = [int(m["label"]) for m in valid_data[i:i+batch_size]]

        labels = torch.tensor(labels).cuda()

        outputs = model(**inputs, labels=labels)

        scores = outputs.logits

        predictions = torch.argmax(scores, dim=1)

        """
        print("GENERAL INFO FOR BATCH "+str(i))
        print("************************************************************")
        print()
        print("scores:",scores)
        print("predictions and labels:",predictions,labels)       
        print("labels:",labels)
        print("preds:",predictions)
        print()
        print("************************************************************")
        """

        print("labels:",labels)
        print("preds:",predictions)

        total+=batch_size

        for k in range(len(labels)):
            if labels[k]==predictions[k]:
                correct+=1
        
    acc = correct / total

    print("Total accuracy:",acc)

    return acc


In [ ]:
def visual_embeds_of_batches(cfg_path, data_dir_train, data_dir_val, batch_size, train_data, valid_data):

  valid_data_fv = valid_data[:int(0.8*len(valid_data))]
  valid_data_ft = valid_data[int(0.8*len(valid_data)):]

  visual_embeds_train_all, visual_embeds_val_all, visual_embeds_val_ft = [],[],[]

  for i in tqdm(np.arange(0, len(valid_data_ft), batch_size)): 
      vt_visual_embeds = get_visual_embeds(cfg_path,data_dir_val,valid_data_ft[i:i+batch_size])
      visual_embeds_val_ft.append(vt_visual_embeds)

  for i in tqdm(np.arange(0, len(train_data), batch_size)):
      t_visual_embeds = get_visual_embeds(cfg_path,data_dir_train,train_data[i:i+batch_size])
      print(t_visual_embeds.shape)
      print(t_visual_embeds[0].shape)
      visual_embeds_train_all.append(t_visual_embeds)
  
  for i in tqdm(np.arange(0, len(valid_data_fv), batch_size)): 
      v_visual_embeds = get_visual_embeds(cfg_path,data_dir_val,valid_data_fv[i:i+batch_size])
      print(v_visual_embeds.shape)
      print(v_visual_embeds[0].shape)
      visual_embeds_val_all.append(v_visual_embeds)

  return valid_data_ft, valid_data_fv, visual_embeds_train_all, visual_embeds_val_ft, visual_embeds_val_all
    

In [ ]:
data_dir_train = data_dir_vsr
data_dir_val = data_dir_c3d
batch_size = 16
train_data = train_data_vsr
valid_data = valid_data_c3d
valid_ft, valid_fv, te, ve_ft, ve_fv = visual_embeds_of_batches(cfg_path, data_dir_train, data_dir_val, batch_size, train_data, valid_data)

  0%|          | 0/1 [00:00<?, ?it/s]

  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


(1024, 1024, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
img shape: torch.Size([16, 3, 416, 416])


/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


proposals: 16
features: 4
features elem: torch.Size([16, 256, 104, 104])
prop: 16
after model roi heads: torch.Size([16000, 256, 7, 7])
torch.Size([16000, 256, 7, 7])
torch.Size([16000, 12544])
torch.Size([16000, 1024])
torch.Size([16000, 1024])
torch.Size([16000, 1024])


...

(1024, 1024, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
img shape: torch.Size([16, 3, 416, 416])
proposals: 16
features: 4
features elem: torch.Size([16, 256, 104, 104])
prop: 16
after model roi heads: torch.Size([16000, 256, 7, 7])
torch.Size([16000, 256, 7, 7])
torch.Size([16000, 12544])
torch.Size([16000, 1024])
torch.Size([16000, 1024])
torch.Size([16000, 1024])
torch.Size([16, 100, 1024])
torch.Size([100, 1024])


  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


(1024, 1024, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
img shape: torch.Size([16, 3, 416, 416])
proposals: 16
features: 4
features elem: torch.Size([16, 256, 104, 104])
prop: 16
after model roi heads: torch.Size([16000, 256, 7, 7])
torch.Size([16000, 256, 7, 7])
torch.Size([16000, 12544])
torch.Size([16000, 1024])
torch.Size([16000, 1024])
torch.Size([16000, 1024])
torch.Size([16, 100, 1024])
torch.Size([100, 1024])


  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


(1024, 1024, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
(400, 400, 3)
img shape: torch.Size([16, 3, 416, 416])
proposals: 16
features: 4
features elem: torch.Size([16, 256, 104, 104])
prop: 16
after model roi heads: torch.Size([16000, 256, 7, 7])
torch.Size([16000, 256, 7, 7])
torch.Size([16000, 12544])
torch.Size([16000, 1024])
torch.Size([16000, 1024])
torch.Size([16000, 1024])
torch.Size([16, 100, 1024])
torch.Size([100, 1024])


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = VisualBertForVisualReasoning.from_pretrained("uclanlp/visualbert-nlvr2-coco-pre")
visualbert_train(model, train_data, valid_ft, te, ve_ft, batch_size)

Some weights of the model checkpoint at uclanlp/visualbert-nlvr2-coco-pre were not used when initializing VisualBertForVisualReasoning: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing VisualBertForVisualReasoning from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VisualBertForVisualReasoning from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VisualBertForVisualReasoning were 

  0%|          | 0/5 [00:00<?, ?it/s]

Loss for batch 0: 0.653840184211731


  0%|          | 0/1 [00:00<?, ?it/s]

labels: tensor([1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0], device='cuda:0')
preds: tensor([0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0], device='cuda:0')
Total accuracy: 0.5625
New accuracy value for validation batch for train: 0.5625
NEW BEST ACCURACY: 0.5625
Loss for batch 1: 0.6764118075370789


...

labels: tensor([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0], device='cuda:0')
preds: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Total accuracy: 0.6875
New accuracy value for validation batch for train: 0.6875
Loss for batch 3: 0.731433629989624


  0%|          | 0/1 [00:00<?, ?it/s]

labels: tensor([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0], device='cuda:0')
preds: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Total accuracy: 0.6875
New accuracy value for validation batch for train: 0.6875
Loss for batch 4: 0.7274218201637268


  0%|          | 0/1 [00:00<?, ?it/s]

labels: tensor([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0], device='cuda:0')
preds: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Total accuracy: 0.6875
New accuracy value for validation batch for train: 0.6875
LAST BEST ACCURACY: 0.6875


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
saved_model = VisualBertForVisualReasoning.from_pretrained("/content/gdrive/My Drive/Colab Notebooks/copy_rp/tmp/best_checkpoint")
visualbert_eval(saved_model, valid_fv, ve_fv, batch_size)

  0%|          | 0/4 [00:00<?, ?it/s]

labels: tensor([1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
preds: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
labels: tensor([0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
preds: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
labels: tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
preds: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
labels: tensor([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], device='cuda:0')
preds: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
Total accuracy: 0.890625


0.890625

  0%|          | 0/4 [00:00<?, ?it/s]

labels: tensor([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0], device='cuda:0')
preds: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
labels: tensor([1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1], device='cuda:0')
preds: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
labels: tensor([0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1], device='cuda:0')
preds: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
labels: tensor([0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1], device='cuda:0')
preds: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
Total accuracy: 0.4375


0.4375

In [ ]:
#print(outputs["instances"].pred_classes)
#print(outputs["instances"].pred_boxes)